In [ ]:
!pip install roughviz

In [9]:
# importing required modules
from math import  *
import PyPDF2 
from bs4 import BeautifulSoup as bs
import lxml.etree as etree
import pandas as pd
import copy

In [10]:
sample_pdf='' #this is the pdf with links
xml_main='' #this contains the page locations
annot_xml='' #this contains annotations width

In [14]:
def compute_skip(xml_main,sample_pdf):
    """decides if there exists an extra page in the header defining the class labels"""
    # creating a pdf file object  
    pdfFileObj = open(sample_pdf, 'rb')  

    # creating a pdf reader object  
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

    true_page_count=pdfReader.numPages

    pdfFileObj.close() 
    
    with open(xml_main, "r") as file:
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        #print(content)
        content = "".join(content)
        bs_content = bs(content, "xml")
        file.close()
        
    pages=bs_content.find_all("Page")
    if(true_page_count==len(pages)):
        return False,pages
    else:
        #remove the first page info
        pages.pop(0)
        return True,pages
    
    
def get_all_boxes(xml,skip=False):
    """returns the annotation boxes from the file"""
    # Read the XML file
    with open(xml, "r") as file:
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        #print(content)
        content = "".join(content)
        bs_content = bs(content, "xml")
        
        page_info=bs_content.find_all("ANNOTATION")
        quads=bs_content.find_all("QUADRILATERAL")
        
        
        
        extractions=[]
        #filter magnifications for certain pages
        
        
        for action_,quad in zip(page_info,quads):
            if(action_.DEST.string!=None): #valid box
                page_no=int(action_.get("pagenum")) #basically the page_num
                
                class_=action_.DEST.string #the type of the box
                
                #if the type of the box is some link then skip it and dont extract it
                if("@" in class_):
                    continue
                
                
                coord=[[float(x.get("HPOS")),float(x.get("VPOS"))] for x in quad] #the coordinates
                if(skip):
                    extractions.append([page_no-1,coord,class_]) 
                else:
                    extractions.append([page_no,coord,class_]) 
    
    file.close()
                
    return extractions

def get_page_wise(pages):
    """extract all page_wise information as textline"""
    page_wise_text={}
    for i in range(len(pages)):
        txt_lines=pages[i].find_all("TextLine")
        lines=[]
        for line in txt_lines:
            text=""
            
            hpos=round(float(line.get("HPOS")))
            vpos=round(float(line.get("VPOS")))
            height=round(float(line.get("HEIGHT")))
            width=round(float(line.get("WIDTH")))
            
            top_left=[hpos,vpos]
            bot_right=[hpos+height,vpos+width]
            top_right=[hpos,vpos+width]
            bot_left=[hpos+height,width]
            
            coordinates=[bot_left,top_left,bot_right,top_right]
            
            
            for second in line:
                if(str(second).startswith("<String")): #its a string 
                    try:
                        content=second.get("CONTENT")
                        #print(content)
                        #we can also extract the font 

                        text+=content
                    except:
                        continue


                if(str(second).startswith("<SP")):
                    text+=" " #add a space

            lines.append([text,coordinates])

        page_wise_text[i+1]=lines
        
    return page_wise_text

def box_to_text_using_xml(xml_main,boxes,show_text=False,skip=False,margin=20):
    """looks for the textblocks that exist in the annotations 
    finds the text inside the boxes that are annotated
    """
    with open(xml_main,"r") as file:
        content=file.readlines()
        content = "".join(content)
        bs_content = bs(content, "xml")
        #print(bs_content)
        text_blocks={}
        page_info=bs_content.find_all("Page") #read all the page based information
        
        if(skip): #compute the skipping for pdf's that have a header page explaining the category labels
            page_info.pop(0)
        
        
        for i,page in enumerate(page_info): #traverse through all pages
            
            
                
            page_text=page.find_all("TextLine") #look for all textlines and 
            text=[]
            for text_bl in page_text: #for text line in all textlines
    
                for element in text_bl: #for elements inside a text line
                    if(str(element).startswith("<String")): #its a string 
                        try:
                            content=element.get("CONTENT")
                            #print(content)
                            h_pos=floor(float(element.get("HPOS")))
                            v_pos=floor(float(element.get("VPOS")))
                            #we can also extract the font 

                            text.append([content,h_pos,v_pos])
                        except:
                            print(element)
                            print(i)
                            print(magnifications[i][0])
                            print(content)
                        
                        
                    if(str(element).startswith("<SP")):
                        val=text.pop()
                        val[0]+=" "
                        text.append(val)
                        
                #print(text)
                
            #now we have text containing information about all the words
            
            text_blocks[i+1]=text
    
        file.close()
        
        
    #print(text_blocks) #this is actually the text words per page
    #print(text_blocks[3])
    
    box_text={}
    string_cats=[]
    for box in boxes: #basically the big box
        all_chars_on_page=text_blocks[box[0]] #small boxes
        

        #coordinates of the big box
        top_left_bb= copy.deepcopy(box[1][1])
        bot_right_bb=copy.deepcopy(box[1][2])


        #print(all_chars_on_page)
        string_cat=""
        
        for chars in all_chars_on_page:
            
            
            
            if(top_left_bb[0]-margin<=chars[1]<=bot_right_bb[0]-margin and top_left_bb[1]<=chars[2]<=bot_right_bb[1]):
                string_cat+=chars[0]
                    
        string_cats.append((box[0],string_cat))
        
    #print(string_cats)

    for element in string_cats:
        if(element[0] not in box_text):
            box_text[element[0]]=[element[1]]
        else:
            box_text[element[0]].append(element[1])
            
    return box_text

def add_tags_to_interesting_areas(boxes,interesting_regions):
    """add the tags to thresholds"""
    cnt=[] #get basically all the text lines
    for element in interesting_regions.values():
        for sub_ele in element:
            cnt.append(sub_ele)


    #merge boxes with extracted text 
    boxes_with_text=[] #add the text to the coordinates
    for box,char in zip(boxes,cnt):
        boxes_with_text.append(box+[char])
    
    def filter_numbers(val): #simply numbers dont make sense either
        try:
            int_number=int(val.strip())
            return False
        except:
            return True

    #basically not a number
    filtered_boxes_with_text=list(filter(lambda x: filter_numbers(x[3]),boxes_with_text)) #filter everything that has the text section more than 1
    
    # again make interesting regions but this time it does not contains the boxes and contains tags
    interesting_regions={}  
    for box in filtered_boxes_with_text:
        page=box[0]
        if(page not in interesting_regions):
            interesting_regions[page]=[box[1:]]
        else:
            interesting_regions[page].append(box[1:])
            
    return interesting_regions

def filter_blanks(interesting_regions):
    for k,v in interesting_regions.items():
        interesting_regions[k]=list(filter(lambda x:len(x[2])>0,interesting_regions[k]))
    return interesting_regions

def make_data_frame_all_data(page_wise_text):
    texts=[]
    pages=[]
    coordinates=[]
    
    for k,element in page_wise_text.items():
        for text in element:
            texts.append(text[0])
            pages.append(k)
            coordinates.append(text[1])
            
    temp=[texts,pages,coordinates]
    
    df=pd.DataFrame(temp).T
    #add another column of labels
    
    
    df.columns=["texts","page_no","coordinates"]
    df["labels"]="Basic"
    
    return df

#make dataframe out of the page wise text
#and then if the entry exist in df then add another section of the label
def make_data_frame_interesting(interesting_regions):
    text=[]
    tag=[]
    pages=[]
    coords=[]
    
    for k,v in interesting_regions.items():
        for sub_element in v:
            tag.append(sub_element[1])
            text.append(sub_element[2])
            pages.append(k)
            coords.append(sub_element[0])
    dict={'texts': text, 'labels': tag, "page_no" :pages, "coordinates" :coords} 

    df = pd.DataFrame(dict)
    
    return df

#traverse through each annotation in the annotations and look if the line lies under the annotation box
def annotate_all(all_df,page_wise_text,interesting_regions,margin=14):

    main_list=[]
    for page_no in page_wise_text:
        if(page_no not in interesting_regions):
            for element in page_wise_text[page_no]:
                element.append("Basic")
                element.append(page_no)
                main_list.append(element)
        else:
            values= interesting_regions[page_no]

            all_coordinates_annotations=list(map(lambda x: x[:3] ,values))
            #print(all_coordinates)
            #traverse through each line coordinates in the coordinates
            #filter specific page
            test=all_df[all_df["page_no"]==page_no]

            probable=[]
            all_line_coordinates = test[["texts", "coordinates"]].values.tolist()
            for annotation_box in all_coordinates_annotations:
                for line_to_check in all_line_coordinates:
                    match=False
                    #print("="*20)
                    #print(annotation_box)
                    #print("="*20)
                    #print(line_to_check)

                    top_left_annotation=annotation_box[0][1]
                    bot_right_annotation=annotation_box[0][2]

                    top_left_line=line_to_check[1][1]
                    bot_right_line=line_to_check[1][2]

                    #print(top_left_annotation,bot_right_annotation)
                    #print(top_left_line,bot_right_line)

                    if(top_left_annotation[0]-margin<=top_left_line[0]<=bot_right_annotation[0] and 
                       top_left_annotation[1]<=top_left_line[1]<=bot_right_annotation[1]):
                        match=True
                        if(match==True):
                            #print("+"*20)
                            #print(line_to_check,annotation_box[0],annotation_box[1],page_no)
                            probable.append([line_to_check[0],annotation_box[0],annotation_box[1],page_no])
                            match=False

            tmp=[element[0] for element in probable]
            labels=[element[2] for element in probable]
            #print(len(labels),len(tmp))

            for element in all_line_coordinates:
                if(element[0] in tmp):
                    #print("="*20)
                    ind=tmp.index(element[0])
                    element.append(labels[ind])
                    element.append(page_no)
                    #print(element)
                    main_list.append(element)
                    #print("="*20)
                else:
                    element.append("Basic")
                    element.append(page_no)
                    main_list.append(element)
                    #print(element)

    df1=pd.DataFrame(main_list,columns=["texts", "coordinates", "labels", "page_no"])
    return df1

    
skip,pages=compute_skip(xml_main,sample_pdf)
boxes=get_all_boxes(annot_xml,skip=skip) #coordinates of interesting regions from annotations xml
page_wise_text=get_page_wise(pages) #all the text present as textlines from the xml main
interesting_regions=box_to_text_using_xml(xml_main,boxes,show_text=False,skip=skip) #gets all the text for annotation boxes
interesting_regions=add_tags_to_interesting_areas(boxes,interesting_regions) #assign tags to interesting regions

#filtering blank spaces to
#==========================================================
interesting_regions=filter_blanks(interesting_regions)
#==========================================================

all_df=make_data_frame_all_data(page_wise_text) #make a dataframe out of all lines
df=make_data_frame_interesting(interesting_regions) #make a dataframe out of intersting regions
df.to_csv("test.csv") #save if necessary

df1=annotate_all(all_df,page_wise_text,interesting_regions)

print("="*20+"Extracting from XML file"+"="*20)
print("Total no of pages:",len(pages))
print("Skip present:",skip)
print("All boxes extracted:",sum([len(element) for element in page_wise_text.values()]))
print("\n")
print("="*20+"showing the first few lines of the first page to show extraction"+"="*20)
for element in page_wise_text[1][:7]:
    print(element)
print("\n")
print("="*20+"for each annotation comparing the text blocks that fit in"+"="*20)
print("Pages with annotations:",list(interesting_regions.keys()))
temp=list(interesting_regions.keys())[0]
print("Some of the lines on the page {} are:".format(temp))
for element in interesting_regions[temp][:7]:
    print(element[2])

print("="*20+"all lines in the pdf (in a dataframe)"+"="*20)
display(all_df)

print("="*20+"annotated lines in the pdf (in a dataframe)"+"="*20)
display(df)

print("="*20+"in sequence dataframe"+"="*20)
display(df1)

#df1.to_csv("test.csv") #save if necessary

====================Extracting from XML file====================
Total no of pages: 26
Skip present: True
All boxes extracted: 1290


====================showing the first few lines of the first page to show extraction====================
['Unifying computational entropies', [[206, 230], [191, 122], [206, 352], [191, 352]], 'Basic', 1]
['via Kullback–Leibler divergence', [[211, 219], [196, 144], [211, 363], [196, 363]], 'Basic', 1]
['Rohit Agrawal ∗', [[190, 83], [180, 176], [190, 259], [180, 259]], 'Basic', 1]
['rohitagr@seas.harvard.edu', [[153, 154], [143, 193], [153, 347], [143, 347]], 'Basic', 1]
['Yi-Hsiu Chen †', [[372, 77], [362, 176], [372, 253], [362, 253]], 'Basic', 1]
['yhchen@seas.harvard.edu', [[338, 141], [328, 193], [338, 334], [328, 334]], 'Basic', 1]
['Thibaut Horel ‡', [[189, 81], [179, 210], [189, 291], [179, 291]], 'Basic', 1]


====================for each annotation comparing the text blocks that fit in====================
Pages with annotations: [3, 4, 5, 6, 7, 

,texts,page_no,coordinates,labels
0,Unifying computational entropies,1,"[[206, 230], [191, 122], [206, 352], [191, 352]]",Basic
1,via Kullback–Leibler divergence,1,"[[211, 219], [196, 144], [211, 363], [196, 363]]",Basic
2,Rohit Agrawal ∗,1,"[[190, 83], [180, 176], [190, 259], [180, 259]]",Basic
3,rohitagr@seas.harvard.edu,1,"[[153, 154], [143, 193], [153, 347], [143, 347]]",Basic
4,Yi-Hsiu Chen †,1,"[[372, 77], [362, 176], [372, 253], [362, 253]]",Basic
...,...,...,...,...
1285,The witness hardness in relative entropy then ...,26,"[[103, 425], [93, 344], [103, 769], [93, 769]]",Basic
1286,2−x is convex) and the witness hardness in δ-m...,26,"[[103, 425], [93, 355], [103, 780], [93, 780]]",Basic
1287,inequality by considering the event that the s...,26,"[[104, 425], [94, 371], [104, 796], [94, 796]]",Basic
1288,event has density at least δ).,26,"[[104, 137], [94, 384], [104, 521], [94, 521]]",Basic


====================annotated lines in the pdf (in a dataframe)====================


,texts,labels,page_no,coordinates
0,"Deﬁnition 1.1 (next-block pseudoentropy, infor...",uri:theorem.Definition.1,3,"[[92.604, 327.634], [92.604, 314.733], [519.39..."
1,"and X = (X1, . . . , Xm) be a random variable ...",uri:theorem.Definition.1,3,"[[92.604, 341.184], [92.604, 328.282], [519.39..."
2,We say that X has next-block pseudoentropy at ...,uri:theorem.Definition.1,3,"[[92.604, 354.127], [92.604, 342.437], [519.39..."
3,"Z = (Z1, . . . , Zm), jointly distributed with...",uri:theorem.Definition.1,3,"[[92.604, 368.282], [92.604, 355.38], [519.396..."
4,"1. For all i = 1, . . . , m, (X1, . . . , Xi−1...",uri:theorem.Definition.1,3,"[[119.877, 389.82], [119.877, 376.918], [519.3..."
...,...,...,...,...
418,"=Er←Ry←G 1 (r)w← G 2(r)2 − KL ∗r,y,w ( R, G 1 ...",uri:proof.64,26,"[[92.604, 331.922], [92.604, 284.848], [519.39..."
419,The witness hardness in relative entropy then ...,uri:proof.64,26,"[[92.604, 354.787], [92.604, 341.885], [519.39..."
420,2−x is convex) and the witness hardness in δ-m...,uri:proof.64,26,"[[92.604, 368.336], [92.604, 354.842], [519.39..."
421,inequality by considering the event that the s...,uri:proof.64,26,"[[92.604, 381.885], [92.604, 368.983], [519.39..."


====================in sequence dataframe====================


,texts,coordinates,labels,page_no
0,Unifying computational entropies,"[[206, 230], [191, 122], [206, 352], [191, 352]]",Basic,1
1,via Kullback–Leibler divergence,"[[211, 219], [196, 144], [211, 363], [196, 363]]",Basic,1
2,Rohit Agrawal ∗,"[[190, 83], [180, 176], [190, 259], [180, 259]]",Basic,1
3,rohitagr@seas.harvard.edu,"[[153, 154], [143, 193], [153, 347], [143, 347]]",Basic,1
4,Yi-Hsiu Chen †,"[[372, 77], [362, 176], [372, 253], [362, 253]]",Basic,1
...,...,...,...,...
1285,The witness hardness in relative entropy then ...,"[[103, 425], [93, 344], [103, 769], [93, 769]]",uri:proof.64,26
1286,2−x is convex) and the witness hardness in δ-m...,"[[103, 425], [93, 355], [103, 780], [93, 780]]",uri:proof.64,26
1287,inequality by considering the event that the s...,"[[104, 425], [94, 371], [104, 796], [94, 796]]",uri:proof.64,26
1288,event has density at least δ).,"[[104, 137], [94, 384], [104, 521], [94, 521]]",uri:proof.64,26


In [4]:
df1[df1["labels"]!="Basic"]

,texts,coordinates,labels,page_no
301,Fact 6 Let G be a graph and let f (v) ≥ n/g(n)...,"[[81, 337], [72, 441], [81, 778], [72, 778]]",uri:theorem.Fact.1,6
302,"f -factor of G, then the number of components ...","[[82, 296], [72, 453], [82, 749], [72, 749]]",uri:theorem.Fact.1,6
328,Fact 8 Let S be a subset of E(G). An f -factor...,"[[81, 337], [72, 256], [81, 593], [72, 593]]",uri:theorem.Fact.2,7
329,"if one exists, can be computed in polynomial t...","[[81, 199], [72, 269], [81, 468], [72, 468]]",uri:theorem.Fact.2,7
350,Fact 11 Let A be an alternating circuit and S ...,"[[81, 337], [72, 564], [81, 901], [72, 901]]",uri:theorem.Fact.3,7
351,polynomial time algorithm that outputs a set M...,"[[81, 337], [72, 576], [81, 913], [72, 913]]",uri:theorem.Fact.3,7
357,Fact 12 Let H be an f -factor of G and let Q b...,"[[81, 337], [72, 92], [81, 429], [72, 429]]",uri:theorem.Fact.4,8
358,If H/Q is connected and H[Q] is connected for ...,"[[81, 337], [72, 104], [81, 441], [72, 441]]",uri:theorem.Fact.4,8
359,f -factor.,"[[82, 34], [72, 116], [82, 150], [72, 150]]",uri:theorem.Fact.4,8
389,Observation 14 Let G be an undirected graph an...,"[[81, 337], [72, 579], [81, 916], [72, 916]]",uri:theorem.Observation.5,8


In [8]:
def nlp_dataframe_extraction(sample_pdf,xml_main,annot_xml):
    
    try:
        skip,pages=compute_skip(xml_main,sample_pdf)
        boxes=get_all_boxes(annot_xml,skip=skip) #coordinates of interesting regions from annotations xml
        page_wise_text=get_page_wise(pages) #all the text present as textlines from the xml main
        interesting_regions=box_to_text_using_xml(xml_main,boxes,show_text=False,skip=skip) #gets all the text for annotation boxes
        interesting_regions=add_tags_to_interesting_areas(boxes,interesting_regions) #assign tags to interesting regions
        
        #removing blank lines can help approximating these blank lines
        #=====================
        interesting_regions=filter_blanks(interesting_regions)
        #=====================
        
        all_df=make_data_frame_all_data(page_wise_text) #make a dataframe out of all lines
        df=make_data_frame_interesting(interesting_regions) #make a dataframe out of intersting regions
        df1=annotate_all(all_df,page_wise_text,interesting_regions)
        df1.to_csv("test.csv")
        return df1

    except FileNotFoundError:
        if(not os.path.exists(xml_main)):
            print("one of the files missing, please check the folder : {}".format(xml_main))
        return 1
    except:
        return 2
    
sample_pdf='/Users/mv96/Desktop/temp/1902.11202/1902.11202.pdf' #this is the pdf with links
xml_main='/Users/mv96/Desktop/temp/1902.11202/1902.11202.xml' #this contains the page locations
annot_xml='/Users/mv96/Desktop/temp/1902.11202/1902.11202_annot.xml' #this contains annotations width

res=nlp_dataframe_extraction(sample_pdf,xml_main,annot_xml)
res.to_csv("test.csv")

In [ ]:
pdf_directory_6000="/Users/mv96/tkb_pdfs"

import os
import functools
arr = os.listdir(pdf_directory_6000)


#go in each folder
from os import walk

#getting the file names of xml files and pdf's
paths=[]
for (dirpath, dirnames, filenames) in walk(pdf_directory_6000):
    for sub_folder in dirnames:
        for (dirpath, dirnames, filenames) in walk(pdf_directory_6000+"/"+sub_folder):
            pdf_path=pdf_directory_6000+"/"+sub_folder+"/{}.pdf".format(sub_folder)
            xml_main=pdf_directory_6000+"/"+sub_folder+"/{}.xml".format(sub_folder)
            annot_xml=pdf_directory_6000+"/"+sub_folder+"/{}_annot.xml".format(sub_folder)
            if([pdf_path,xml_main,annot_xml] not in paths):
                paths.append([pdf_path,xml_main,annot_xml])



In [ ]:
#multi process the code
from joblib import Parallel, delayed  
from tqdm import tqdm

try:
    #there could be over flow of resources so one can reduce the n_jobs parameter
    res = Parallel(n_jobs=4, verbose=10)(delayed(nlp_dataframe_extraction)(file[0],file[1],file[2])
                                 for file in tqdm(paths))


    res=list(filter(lambda x: isinstance(x,pd.DataFrame),res))

    res=functools.reduce(lambda a,b : pd.concat([a,b]),res)
    
except:
    print("couldn't run multiprocessing")

In [ ]:
#sequential execution
res=[]
for file in tqdm(paths):
    df=nlp_dataframe_extraction(file[0],file[1],file[2])
    res.append(df)
    
res=list(filter(lambda x: isinstance(x,pd.DataFrame),res))

res=functools.reduce(lambda a,b : pd.concat([a,b]),res)

In [ ]:
res.shape

In [ ]:
res.head()

In [ ]:
res.to_csv("finalv2.csv")

In [1]:
import pandas as pd

df=pd.read_csv("finalv2.csv")
df=df.drop(columns=["Unnamed: 0"])
df=df.dropna(subset=['labels'])

def round_labels(val):
    
    if("theorem" in val):
        return "Theorem"
    elif ("proof" in val):
        return "Proof"
    else:
        return "Basic"
    
df["labels"]=df["labels"].apply(round_labels)


/Users/mv96/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df.shape

(5281411, 4)

In [3]:
df["labels"].value_counts()

Basic      3199559
Proof      1582665
Theorem     499187
Name: labels, dtype: int64

In [4]:
df=df.drop_duplicates(subset=["texts"])
df["labels"].value_counts()

Basic      2349559
Proof      1077553
Theorem     354985
Name: labels, dtype: int64

In [5]:
df.shape

(3782097, 4)

In [7]:
categories=list(df["labels"].value_counts().index)
vals=list(df["labels"].value_counts())
import roughviz
import pandas as pd 
d = {'labels': categories, 'values': vals}
df_ = pd.DataFrame(data=d)
roughviz.bar(df_["labels"], df_["values"], axisRoughness = 0.7, axisStrokeWidth = 0.7, roughness=2.3, highlight="gray")

In [8]:
roughviz.pie(df_["labels"], df_["values"], axisRoughness = 0.7, axisStrokeWidth = 0.7, roughness=2.3, highlight="gray")

# Balancing the dataset by undersampling

In [45]:
#shuffle the rows
df=df.sample(frac=1,random_state=42)
df.shape

(3782097, 4)

In [46]:
#shuffle all the rows startified sample
stratified=True
def stratified_sample_df(df, col, n_samples):
        n = min(n_samples, df[col].value_counts().min())
        df_ = df.groupby(col).apply(lambda x: x.sample(n))
        df_.index = df_.index.droplevel(0)
        return df_

if(stratified==True):
    new=stratified_sample_df(df,"labels",354985)

In [47]:
#new.to_csv("for_transformers.csv")

In [48]:
counts_of_occurances=list(new["labels"].value_counts().values)
labels=list(new["labels"].value_counts().index)

d = {'labels': labels, 'values': counts_of_occurances}
df_ = pd.DataFrame(data=d)
roughviz.bar(df_["labels"], df_["values"], axisRoughness = 0.7, axisStrokeWidth = 0.7, roughness=2.3, highlight="gray")



In [49]:
roughviz.pie(df_["labels"], df_["values"], axisRoughness = 0.7, axisStrokeWidth = 0.7, roughness=2.3, highlight="gray")

# Full preprocessed data

In [102]:
#if you dont want to use full data then  that to happen
new=df

In [103]:
new.shape

(3782097, 4)

In [104]:
#train test split and random splitting
from sklearn.model_selection import train_test_split
# create training and testing data (80% train and 20% test)
data_train, data_test = train_test_split(new,test_size=0.2,random_state=42,stratify=new["labels"])#test size here relates to second var i.e test

print("="*20+" train data (80%)"+"="*20)
print (data_train.shape)
print("="*20+" test data (20%)"+"="*20)
print (data_test.shape)

==================== train data (80%)====================
(3025677, 4)
==================== test data (20%)====================
(756420, 4)


In [105]:
#add label encoding to the data 
x_train , y_train = data_train["texts"],data_train["labels"]
x_test , y_test = data_test["texts"],data_test["labels"]

In [106]:
#### label encoding my y of train and test data 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

print("The classes obtained")
print(le.classes_)

y_train=le.transform(y_train)
y_test=le.transform(y_test)

x_train=x_train.to_list()
x_test=x_test.to_list()

The classes obtained
['Basic' 'Proof' 'Theorem']


In [107]:
import numpy as np
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(3025677,) (3025677,) (756420,) (756420,)


In [108]:
x_train=pd.DataFrame(x_train,columns=["text"])
y_train=pd.DataFrame(y_train,columns=["labels"])
a=pd.concat([x_train,y_train],axis=1)

In [109]:
a0=a[a["labels"]==0]
a1=a[a["labels"]==1]
a2=a[a["labels"]==2]

In [110]:
temp=pd.concat([a0.iloc[:1000000],a1,a2]) #select only first 10000000 for memory issues on colab
x_train=temp["text"]
y_train=temp["labels"]


In [111]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2146030,)
(2146030,)
(756420,)
(756420,)
